<a href="https://colab.research.google.com/github/chadeowen/DS-Unit-2-Sprint-3-Advanced-Regression/blob/master/Copy_of_LS_DS1_231_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science - Logistic Regression

Logistic regression is the baseline for classification models, as well as a handy way to predict probabilities (since those too live in the unit interval). While relatively simple, it is also the foundation for more sophisticated classification techniques such as neural networks (many of which can effectively be thought of as networks of logistic models).

## Lecture - Where Linear goes Wrong
### Return of the Titanic 🚢

You've likely already explored the rich dataset that is the Titanic - let's use regression and try to predict survival with it. The data is [available from Kaggle](https://www.kaggle.com/c/titanic/data), so we'll also play a bit with [the Kaggle API](https://github.com/Kaggle/kaggle-api).

In [0]:
!pip install kaggle

In [0]:
# Note - you'll also have to sign up for Kaggle and authorize the API
# https://github.com/Kaggle/kaggle-api#api-credentials

# This essentially means uploading a kaggle.json file
# For Colab we can have it in Google Drive
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

# You also have to join the Titanic competition to have access to the data
!kaggle competitions download -c titanic

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 59.5MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 28.9MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 3.07MB/s]


In [0]:
# How would we try to do this with linear regression?
import pandas as pd

train_df = pd.read_csv('train.csv').dropna()
test_df = pd.read_csv('test.csv').dropna()  # Unlabeled, for Kaggle submission

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S


In [0]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000
mean,455.366120,0.672131,1.191257,35.674426,0.464481,0.475410,78.682469
std,247.052476,0.470725,0.515187,15.643866,0.644159,0.754617,76.347843
min,2.000000,0.000000,1.000000,0.920000,0.000000,0.000000,0.000000
25%,263.500000,0.000000,1.000000,24.000000,0.000000,0.000000,29.700000
50%,457.000000,1.000000,1.000000,36.000000,0.000000,0.000000,57.000000
75%,676.000000,1.000000,1.000000,47.500000,1.000000,1.000000,90.000000
max,890.000000,1.000000,3.000000,80.000000,3.000000,4.000000,512.329200


In [0]:
from sklearn.linear_model import LinearRegression

X = train_df[['Pclass', 'Age', 'Fare']]
y = train_df.Survived

linear_reg = LinearRegression().fit(X, y)
linear_reg.score(X, y)

0.08389810726550917

In [0]:
linear_reg.predict(test_df[['Pclass', 'Age', 'Fare']])

array([0.79543117, 0.58610823, 0.67595121, 0.793829  , 0.62090522,
       0.72542107, 0.59848968, 0.58734245, 0.48567063, 0.77627736,
       0.84211887, 0.57052439, 0.7754689 , 0.96621114, 0.70287941,
       0.57673837, 0.72321391, 0.75894755, 0.77968041, 0.50246003,
       0.49858077, 0.7474959 , 0.3542282 , 0.61648435, 0.71300224,
       0.66294608, 0.53175333, 0.77397395, 0.68419387, 0.68395536,
       0.52041202, 0.56814038, 0.79586606, 0.81372012, 0.61068545,
       0.57260627, 0.52525981, 0.58055388, 0.45584728, 0.67976208,
       0.8226707 , 0.84286197, 0.96189157, 0.66724612, 0.68589478,
       0.61846513, 0.63455044, 0.68275686, 0.65738372, 0.45198998,
       0.59988596, 0.63845908, 0.63132487, 0.7888473 , 0.60126246,
       0.79714045, 0.78713803, 0.54643775, 0.42823635, 0.7711724 ,
       0.53552976, 0.55608044, 0.54480459, 0.57031915, 0.65080369,
       0.77958926, 0.6371013 , 0.70993488, 0.71493598, 0.60375943,
       0.54407206, 0.48186138, 0.76576089, 0.75456305, 0.53968

In [0]:
linear_reg.coef_

array([-0.08596295, -0.00829314,  0.00048775])

In [0]:
import numpy as np

test_case = np.array([[1, 5, 500]])  # Rich 5-year old in first class
linear_reg.predict(test_case)

array([1.14845883])

In [0]:
np.dot(linear_reg.coef_, test_case.reshape(-1, 1)) + linear_reg.intercept_

array([1.14845883])

In [0]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression().fit(X, y)
log_reg.score(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7103825136612022

In [0]:
log_reg.predict(test_df[['Pclass', 'Age', 'Fare']])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [0]:
log_reg.predict(test_case)[0]

1

In [0]:
help(log_reg.predict)

Help on method predict in module sklearn.linear_model.base:

predict(X) method of sklearn.linear_model.logistic.LogisticRegression instance
    Predict class labels for samples in X.
    
    Parameters
    ----------
    X : array_like or sparse matrix, shape (n_samples, n_features)
        Samples.
    
    Returns
    -------
    C : array, shape [n_samples]
        Predicted class label per sample.



In [0]:
log_reg.predict_proba(test_case)[0]

array([0.02485552, 0.97514448])

In [0]:
# What's the math?
log_reg.coef_

array([[-0.0455017 , -0.02912513,  0.0048037 ]])

In [0]:
log_reg.intercept_

array([1.45878264])

In [0]:
# The logistic sigmoid "squishing" function, implemented to accept numpy arrays
def sigmoid(x):
  return 1 / (1 + np.e**(-x))

In [0]:
sigmoid(log_reg.intercept_ + np.dot(log_reg.coef_, np.transpose(test_case)))

array([[0.97514448]])

So, clearly a more appropriate model in this situation! For more on the math, [see this Wikipedia example](https://en.wikipedia.org/wiki/Logistic_regression#Probability_of_passing_an_exam_versus_hours_of_study).

For live - let's tackle [another classification dataset on absenteeism](http://archive.ics.uci.edu/ml/datasets/Absenteeism+at+work) - it has 21 classes, but remember, scikit-learn LogisticRegression automatically handles more than two classes. How? By essentially treating each label as different (1) from some base class (0).

In [0]:
# Live - let's try absenteeism!
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/00445/Absenteeism_at_work_AAA.zip
!unzip Absenteeism_at_work_AAA.zip	

--2019-01-22 17:30:51--  http://archive.ics.uci.edu/ml/machine-learning-databases/00445/Absenteeism_at_work_AAA.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.249
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.249|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66136 (65K) [application/zip]
Saving to: ‘Absenteeism_at_work_AAA.zip’

Absenteeism_at_work 100%[===================>]  64.59K   179KB/s    in 0.4s    

2019-01-22 17:30:52 (179 KB/s) - ‘Absenteeism_at_work_AAA.zip’ saved [66136/66136]

Archive:  Absenteeism_at_work_AAA.zip
  inflating: Absenteeism_at_work.arff  
  inflating: Absenteeism_at_work.csv  
  inflating: Absenteeism_at_work.xls  
  inflating: Attribute Information.docx  
  inflating: UCI_ABS_TEXT.docx       


In [0]:
!head Absenteeism_at_work.csv

ID;Reason for absence;Month of absence;Day of the week;Seasons;Transportation expense;Distance from Residence to Work;Service time;Age;Work load Average/day ;Hit target;Disciplinary failure;Education;Son;Social drinker;Social smoker;Pet;Weight;Height;Body mass index;Absenteeism time in hours
11;26;7;3;1;289;36;13;33;239.554;97;0;1;2;1;0;1;90;172;30;4
36;0;7;3;1;118;13;18;50;239.554;97;1;1;1;1;0;0;98;178;31;0
3;23;7;4;1;179;51;18;38;239.554;97;0;1;0;1;0;0;89;170;31;2
7;7;7;5;1;279;5;14;39;239.554;97;0;1;2;1;1;0;68;168;24;4
11;23;7;5;1;289;36;13;33;239.554;97;0;1;2;1;0;1;90;172;30;2
3;23;7;6;1;179;51;18;38;239.554;97;0;1;0;1;0;0;89;170;31;2
10;22;7;6;1;361;52;3;28;239.554;97;0;1;1;1;0;4;80;172;27;8
20;23;7;6;1;260;50;11;36;239.554;97;0;1;4;1;0;0;65;168;23;4
14;19;7;2;1;155;12;14;34;239.554;97;0;1;2;1;0;0;95;196;25;40


In [0]:
absent_df = pd.read_table('Absenteeism_at_work.csv', sep=';')
absent_df.head()

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,...,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,4
1,36,0,7,3,1,118,13,18,50,239.554,...,1,1,1,1,0,0,98,178,31,0
2,3,23,7,4,1,179,51,18,38,239.554,...,0,1,0,1,0,0,89,170,31,2
3,7,7,7,5,1,279,5,14,39,239.554,...,0,1,2,1,1,0,68,168,24,4
4,11,23,7,5,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,2


In [0]:
absent_df.shape

(740, 21)

In [0]:
LogisticRegression

sklearn.linear_model.logistic.LogisticRegression

In [0]:
absent_df.columns

Index(['ID', 'Reason for absence', 'Month of absence', 'Day of the week',
       'Seasons', 'Transportation expense', 'Distance from Residence to Work',
       'Service time', 'Age', 'Work load Average/day ', 'Hit target',
       'Disciplinary failure', 'Education', 'Son', 'Social drinker',
       'Social smoker', 'Pet', 'Weight', 'Height', 'Body mass index',
       'Absenteeism time in hours'],
      dtype='object')

In [0]:
X = absent_df.drop('Reason for absence', axis='columns')
y = absent_df['Reason for absence']

absent_log1 = LogisticRegression().fit(X, y)
absent_log1.score(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.5013513513513513

In [0]:
absent_log1

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [0]:
absent_log1.coef_

array([[-1.27508146e-02, -2.20793886e-01, -4.08404457e-02,
         4.22616389e-01,  7.64613632e-03, -6.18658809e-03,
        -3.11523480e-02,  5.52309565e-02, -1.01707258e-02,
        -1.14781041e-01,  2.51984519e+00,  1.41303285e-02,
         3.56702636e-01,  2.27778616e-01,  1.80478520e-01,
         1.33441356e-01, -9.82098540e-03,  5.23702470e-02,
         7.24744784e-02, -3.15221513e+00],
       [ 5.11128074e-02,  1.77166203e-01, -5.01201750e-01,
        -9.34413067e-02,  8.70070989e-03,  2.90683453e-02,
        -8.19496921e-02,  1.30910624e-01,  3.69193210e-03,
         3.58478924e-02, -7.77198534e-01,  9.21419989e-01,
         2.74046853e-01, -6.53744541e-01, -2.74959514e-01,
        -4.46143318e-01,  1.88638790e-01, -7.60409657e-02,
        -6.85430554e-01,  4.55734002e-03],
       [ 1.74477093e-01,  5.91768593e-01,  4.46355493e-02,
         4.29016838e-01,  5.97802716e-02, -2.02281583e-01,
        -4.44906133e-01, -2.65239556e-01,  8.55486140e-03,
        -4.07755000e-01, -6.7

In [0]:
absent_log1.predict(X)

array([23,  0, 23, 25, 23, 23, 22, 28, 18, 25, 28, 28, 28, 23, 23, 28, 23,
       11, 23, 23, 23, 23, 22, 28, 24, 22, 23, 28, 23, 23, 28, 18, 23, 18,
       23, 23, 23, 28, 18, 28, 23, 23, 23, 26, 23, 23, 23, 23, 28, 13,  0,
        0, 23, 23,  0,  0, 23, 28,  0, 23, 28, 28, 28, 23,  0, 23, 28, 23,
       23, 23, 23, 28, 28, 23, 19, 28, 23, 24, 23, 28, 28, 23, 13, 21, 28,
       10, 23, 13, 23, 23, 23, 21, 23, 28, 23, 23, 13, 28, 23, 13, 23, 23,
       23, 28, 13, 22, 23, 28, 23, 22, 23, 28, 23, 23, 27, 27, 18, 23, 27,
       23, 27, 27, 23, 27, 27, 27, 27, 27, 27, 13, 22, 27, 27, 27, 27, 23,
       22, 27, 28, 13, 23, 22, 28, 22,  1,  1, 23, 23, 27, 11, 23, 11, 23,
       23, 22, 28, 28, 28, 17, 13, 28, 13, 23, 13, 11, 18, 13, 18, 22, 17,
       28, 23,  7, 13, 23, 13, 13, 13, 13, 28, 14, 23, 23, 23, 28, 23, 28,
       13, 23, 23, 28, 13, 28, 28, 28, 28, 14, 13, 13, 10, 22, 13, 22,  0,
       13, 28, 28, 13, 13, 28, 28, 28, 13,  0,  0,  0,  0, 28, 13, 22, 13,
       22, 13, 28, 13, 13

In [0]:
absent_log1.predict_proba(X)[0]

array([2.69982649e-06, 6.91615398e-03, 8.57701135e-09, 5.16203311e-08,
       2.18856804e-07, 5.92457902e-08, 2.75125708e-03, 4.07323392e-03,
       1.96525568e-03, 1.28991052e-10, 1.04724148e-02, 2.54302198e-02,
       9.93451673e-05, 3.60953076e-02, 1.02877921e-02, 1.48667629e-04,
       1.20483308e-07, 4.78991713e-21, 2.50188861e-02, 8.82617162e-02,
       2.20255602e-04, 1.37766587e-01, 3.10018914e-01, 1.68568210e-02,
       5.54127149e-02, 1.77084959e-01, 1.38810366e-06, 9.11149514e-02])

In [0]:
?LogisticRegression

## Assignment - real-world classification

We're going to check out a larger dataset - the [FMA Free Music Archive data](https://github.com/mdeff/fma). It has a selection of CSVs with metadata and calculated audio features that you can load and try to use to classify genre of tracks. To get you started:

In [0]:
!wget https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
!unzip fma_metadata.zip

--2019-01-25 15:31:57--  https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
Resolving os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)... 86.119.28.13, 2001:620:5ca1:2ff::ce53
Connecting to os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)|86.119.28.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358412441 (342M) [application/zip]
Saving to: ‘fma_metadata.zip’

fma_metadata.zip    100%[===================>] 341.81M  12.5MB/s    in 27s     

2019-01-25 15:32:29 (12.8 MB/s) - ‘fma_metadata.zip’ saved [358412441/358412441]

Archive:  fma_metadata.zip
 bunzipping: fma_metadata/README.txt  
 bunzipping: fma_metadata/checksums  
 bunzipping: fma_metadata/not_found.pickle  
 bunzipping: fma_metadata/raw_genres.csv  
 bunzipping: fma_metadata/raw_albums.csv  
 bunzipping: fma_metadata/raw_artists.csv  
 bunzipping: fma_metadata/raw_tracks.csv  
 bunzipping: fma_metadata/tracks.csv  
 bunzipping: fma_metadata/genres.csv  
 bunzipping: fma_metadata/raw_echonest.cs

In [0]:
# Thanks A!
import pandas as pd

tracks = pd.read_csv('fma_metadata/tracks.csv', header=[1], index_col=0)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
# Maybe it'd be handy to sample so we have less data so things run faster
tracks = tracks.sample(2000, random_state=42)

In [0]:
help(tracks.sample)

Help on method sample in module pandas.core.generic:

sample(n=None, frac=None, replace=False, weights=None, random_state=None, axis=None) method of pandas.core.frame.DataFrame instance
    Returns a random sample of items from an axis of object.
    
    Parameters
    ----------
    n : int, optional
        Number of items from axis to return. Cannot be used with `frac`.
        Default = 1 if `frac` = None.
    frac : float, optional
        Fraction of axis items to return. Cannot be used with `n`.
    replace : boolean, optional
        Sample with or without replacement. Default = False.
    weights : str or ndarray-like, optional
        Default 'None' results in equal probability weighting.
        If passed a Series, will align with target object on index. Index
        values in weights not found in sampled object will be ignored and
        index values in sampled object not in weights will be assigned
        weights of zero.
        If called on a DataFrame, will accept t

In [0]:
tracks.describe()

,comments,favorites,id,listens,tracks,comments.1,favorites.1,id.1,latitude,longitude,bit_rate,comments.2,duration,favorites.2,interest,listens.1,number
count,2000.000000,2000.000000,2000.000000,2.000000e+03,2000.000000,2000.000000,2000.000000,2000.00000,830.000000,830.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.0000,2000.00000,2000.000000
mean,0.543500,1.346500,12891.541000,4.244437e+04,19.853000,2.178500,33.452500,12134.92450,39.281330,-38.592210,265319.936000,0.036000,279.785500,3.507000,3886.6915,2701.08200,8.168000
std,3.371764,3.451868,6183.597002,2.274153e+05,35.997421,6.920302,108.435792,6774.94956,19.462718,66.764073,65514.796779,0.387013,302.770817,11.525727,10474.1090,7508.55174,14.255749
min,-1.000000,-1.000000,-1.000000,-1.000000e+00,-1.000000,-1.000000,-1.000000,2.00000,-43.532054,-122.676207,-1.000000,0.000000,3.000000,0.000000,7.0000,5.00000,0.000000
25%,0.000000,0.000000,8017.000000,3.218250e+03,7.000000,0.000000,1.000000,6747.25000,39.290385,-81.424640,196102.000000,0.000000,150.000000,0.000000,598.0000,287.00000,2.000000
50%,0.000000,0.000000,13357.500000,8.778000e+03,11.000000,0.000000,5.000000,12270.00000,41.737588,-73.553993,319333.000000,0.000000,218.000000,1.000000,1306.5000,766.50000,5.000000
75%,0.000000,1.000000,18177.250000,2.492450e+04,18.000000,1.000000,16.000000,17922.50000,48.775418,4.815965,320000.000000,0.000000,306.000000,3.000000,3181.0000,2062.25000,9.000000
max,53.000000,55.000000,22904.000000,3.564243e+06,652.000000,79.000000,951.000000,24342.00000,64.928595,174.885971,321434.000000,10.000000,3697.000000,298.000000,203066.0000,93894.00000,248.000000


In [0]:
pd.set_option('display.max_columns', None)  # Unlimited columns
tracks.head()

album                                                     \
         comments         date_created        date_released engineer   
track_id                                                               
87344           0  2013-07-18 20:12:56  2013-07-14 00:00:00      NaN   
114014          0  2015-02-02 11:31:04                  NaN      NaN   
35711           0  2010-09-15 21:51:15                  NaN      NaN   
29297           2  2010-05-02 03:47:37                  NaN      NaN   
120009          0  2015-05-21 12:25:29  2003-11-01 00:00:00      NaN   

                                                                              \
         favorites     id                                        information   
track_id                                                                       
87344            4  14696  <p><strong>BEAM CAMP</strong><br />At <a title...   
114014           0  17578  <p>Sick world. Nothing else to say.</p>\n<p>ht...   
35711            0   7335  <div style="color: #000000; font-family: Verda...   
29297            1   6271  <p><span id="result_box" class="long_text"><sp...   
120009           2  18317  <p><span style="color: #333333; font-family: '...   

                                                \
         listens producer                 tags   
track_id                                         
87344      67696      NaN                   []   
114014      9706      NaN                   []   
35711        773      NaN                   []   
29297      13488      NaN                   []   
120009     10746      NaN  ['tiki', 'exotica']   

                                                                \
                                           title tracks   type   
track_id                                                         
87344     Making Music With Computers, July 2013     14  Album   
114014                         TIENES SIDA (EP2)      4  Album   
35711                          Four Track Attack     23  Album   
29297               Gauchito Gil contra Colocolo     17  Album   
120009                         Two Zombies Later     31  Album   

                       artist                                         \
            active_year_begin      active_year_end associated_labels   
track_id                                                               
87344     1999-01-01 00:00:00  2005-01-01 00:00:00               NaN   
114014                    NaN                  NaN               NaN   
35711                     NaN                  NaN               NaN   
29297                     NaN                  NaN               NaN   
120009                    NaN                  NaN               NaN   

                                                                      \
                                                        bio comments   
track_id                                                               
87344     <p>Hey, what's up? Lemme know if you wanna rap...        5   
114014                                                  NaN        0   
35711     <p>Rapping in a private language he calls cryp...        0   
29297                                                   NaN        0   
120009                                                  NaN        0   

                                                           \
                 date_created favorites     id   latitude   
track_id                                                    
87344     2009-04-14 19:01:09        35   2631  40.058324   
114014    2015-01-14 11:54:09         3  19382        NaN   
35711     2010-09-15 21:52:10         1   8625  19.363656   
29297     2010-05-02 03:50:50         4   7303        NaN   
120009    2015-05-21 12:25:16         0  20164        NaN   

                                                         \
                       location  longitude      members   
track_id                                                  
87344                        NJ -74.405661  

In [0]:
pd.options.mode.use_inf_as_na = True
tracks.isna().sum()

comments                0
date_created           69
date_released         678
engineer             1727
favorites               0
id                      0
information           436
listens                 0
producer             1685
tags                    0
title                  18
tracks                  0
type                  118
active_year_begin    1573
active_year_end      1895
associated_labels    1725
bio                   681
comments.1              0
date_created.1         14
favorites.1             0
id.1                    0
latitude             1170
location              696
longitude            1170
members              1125
name                    0
related_projects     1771
tags.1                  0
website               553
wikipedia_page       1900
split                   0
subset                  0
bit_rate                0
comments.2              0
composer             1938
date_created.2          0
date_recorded        1893
duration                0
favorites.2 

In [0]:
# While exploring, we want to be efficient and focus on what works for now
# Thanks Dan!
garbage_columns = ['id', 'information','comments.1','title','bio',
                  'members','website','wikipedia_page','split','subset',
                  'comments.2','genres','genres_all','information.1','license',
                   'title.1']
tracks2 = tracks.drop(columns=garbage_columns)


In [0]:
tracks2._get_numeric_data()

,comments,favorites,listens,tracks,favorites.1,id.1,latitude,longitude,bit_rate,duration,favorites.2,interest,listens.1,number
43741,-1.0,-1.0,-1.0,-1.0,82.0,8390.0,39.263517,-76.623942,160000.0,167.0,2.0,1957.0,652.0,7.0
31940,0.0,1.0,19141.0,14.0,15.0,7888.0,46.211401,2.209360,320000.0,386.0,1.0,3123.0,2143.0,8.0
50975,3.0,10.0,458874.0,20.0,431.0,11431.0,40.692455,-73.990364,320000.0,181.0,57.0,44135.0,26393.0,0.0
146981,0.0,0.0,19158.0,11.0,110.0,15891.0,58.007359,56.228149,320000.0,308.0,0.0,3491.0,2908.0,10.0
20460,0.0,1.0,3139.0,5.0,6.0,5443.0,41.871940,12.567380,200655.0,48.0,2.0,2070.0,1383.0,1.0
66123,0.0,1.0,4642.0,11.0,8.0,1905.0,45.508670,-73.553993,256000.0,248.0,0.0,514.0,268.0,9.0
54230,1.0,3.0,39825.0,9.0,8.0,11955.0,53.273880,-9.048350,320000.0,200.0,8.0,7107.0,4372.0,7.0
37271,4.0,6.0,76096.0,154.0,0.0,8924.0,NaN,NaN,320000.0,251.0,1.0,1600.0,135.0,0.0
91060,0.0,2.0,13609.0,4.0,21.0,12534.0,53.233299,-6.123610,128000.0,198.0,5.0,4683.0,4189.0,3.0
89881,0.0,2.0,10262.0,16.0,0.0,16907.0,NaN,NaN,320000.0,140.0,0.0,1108.0,854.0,12.0


In [0]:
tracks2.shape

(2000, 36)

In [0]:
tracks2.isna().sum()

comments                0
date_created           69
date_released         678
engineer             1727
favorites               0
listens                 0
producer             1685
tags                    0
tracks                  0
type                  118
active_year_begin    1573
active_year_end      1895
associated_labels    1725
date_created.1         14
favorites.1             0
id.1                    0
latitude             1170
location              696
longitude            1170
name                    0
related_projects     1771
tags.1                  0
bit_rate                0
composer             1938
date_created.2          0
date_recorded        1893
duration                0
favorites.2             0
genre_top            1087
interest                0
language_code        1739
listens.1               0
lyricist             1995
number                  0
publisher            1974
tags.2                  0
dtype: int64

In [0]:
# remove NaNs from genre_top in tracks2

tracks2 = tracks2.dropna(subset=['genre_top'])

In [0]:
# working with numeric only for time purposes

tracks_numeric = tracks2.select_dtypes('number')
tracks_numeric.isnull().sum()

comments         0
favorites        0
listens          0
tracks           0
favorites.1      0
id.1             0
latitude       526
longitude      526
bit_rate         0
duration         0
favorites.2      0
interest         0
listens.1        0
number           0
dtype: int64

In [0]:
tracks_numeric = tracks_numeric.drop(columns=['latitude','longitude'])
tracks_numeric.isna().sum()

comments       0
favorites      0
listens        0
tracks         0
favorites.1    0
id.1           0
bit_rate       0
duration       0
favorites.2    0
interest       0
listens.1      0
number         0
dtype: int64

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y = tracks2['genre_top']
X = tracks_numeric

# Split into train and test groups
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5,
                                                   random_state=42)
log_reg = LogisticRegression(multi_class='ovr',
                            solver='liblinear',
                            max_iter=500).fit(X_train, y_train)
log_reg.score(X_test, y_test)

0.29978118161925604

In [0]:
# find most important features by creating a model of coefficients and 
# determining those with greatest absolute value

df_coefs = pd.DataFrame(log_reg.coef_, 
                           columns = tracks_numeric.columns,
                           index = log_reg.classes_).abs()

df_coefs

,comments,favorites,listens,tracks,favorites.1,id.1,bit_rate,duration,favorites.2,interest,listens.1,number
Blues,0.005402,0.156922,0.000025,0.348604,0.036903,0.000035,2.501043e-06,0.011851,0.085211,0.000317,0.000522,0.116105
Classical,0.000567,0.000997,0.000019,0.006405,0.033166,0.000080,1.714000e-05,0.000624,0.001484,0.000108,0.000126,0.002650
Easy Listening,0.010236,0.002589,0.000266,0.073705,0.019263,0.000242,2.610912e-05,0.007843,0.035559,0.003293,0.004107,0.030882
Electronic,0.000095,0.000048,0.000005,0.007040,0.001360,0.000031,3.168715e-06,0.000141,0.000373,0.000051,0.000065,0.002149
Experimental,0.000860,0.000072,0.000034,0.025102,0.014676,0.000019,4.384890e-06,0.000632,0.003034,0.000194,0.000764,0.013245
Folk,0.001248,0.000216,0.000082,0.008172,0.011283,0.000021,8.064173e-06,0.001295,0.000068,0.000193,0.000051,0.006366
Hip-Hop,0.000110,0.000024,0.000014,0.005270,0.001432,0.000049,6.383069e-07,0.007396,0.000342,0.000012,0.000013,0.000616
Instrumental,0.001196,0.002398,0.000038,0.014141,0.013914,0.000058,1.469349e-05,0.000377,0.001784,0.001163,0.001211,0.014047
International,0.000697,0.000242,0.000022,0.028156,0.029407,0.000014,9.467783e-06,0.000317,0.008144,0.000160,0.000238,0.011100
Jazz,0.004324,0.011884,0.000013,0.068631,0.009600,0.000155,6.238684e-06,0.000663,0.055383,0.000210,0.000053,0.046110


In [0]:
df_coefs.shape

(14, 12)

In [0]:
df_coefs.mean().sort_values()

bit_rate       0.000007
listens        0.000043
id.1           0.000071
interest       0.000793
listens.1      0.000911
comments       0.003719
duration       0.004770
favorites.2    0.015701
favorites      0.015748
favorites.1    0.025233
number         0.033482
tracks         0.053768
dtype: float64

This is the biggest data you've played with so far, and while it does generally fit in Colab, it can take awhile to run. That's part of the challenge!

Your tasks:
- Clean up the variable names in the dataframe
- Use logistic regression to fit a model predicting (primary/top) genre
- Inspect, iterate, and improve your model
- Answer the following questions (written, ~paragraph each):
  - What are the best predictors of genre?
  - What information isn't very useful for predicting genre?
  - What surprised you the most about your results?

*Important caveats*:
- This is going to be difficult data to work with - don't let the perfect be the enemy of the good!
- Be creative in cleaning it up - if the best way you know how to do it is download it locally and edit as a spreadsheet, that's OK!
- If the data size becomes problematic, consider sampling/subsetting
- You do not need perfect or complete results - just something plausible that runs, and that supports the reasoning in your written answers

If you find that fitting a model to classify *all* genres isn't very good, it's totally OK to limit to the most frequent genres, or perhaps trying to combine or cluster genres as a preprocessing step. Even then, there will be limits to how good a model can be with just this metadata - if you really want to train an effective genre classifier, you'll have to involve the other data (see stretch goals).

This is real data - there is no "one correct answer", so you can take this in a variety of directions. Just make sure to support your findings, and feel free to share them as well! This is meant to be practice for dealing with other "messy" data, a common task in data science.

## Resources and stretch goals

- Check out the other .csv files from the FMA dataset, and see if you can join them or otherwise fit interesting models with them
- [Logistic regression from scratch in numpy](https://blog.goodaudience.com/logistic-regression-from-scratch-in-numpy-5841c09e425f) - if you want to dig in a bit more to both the code and math (also takes a gradient descent approach, introducing the logistic loss function)
- Create a visualization to show predictions of your model - ideally show a confidence interval based on error!
- Check out and compare classification models from scikit-learn, such as [SVM](https://scikit-learn.org/stable/modules/svm.html#classification), [decision trees](https://scikit-learn.org/stable/modules/tree.html#classification), and [naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html). The underlying math will vary significantly, but the API (how you write the code) and interpretation will actually be fairly similar.
- Sign up for [Kaggle](https://kaggle.com), and find a competition to try logistic regression with
- (Not logistic regression related) If you enjoyed the assignment, you may want to read up on [music informatics](https://en.wikipedia.org/wiki/Music_informatics), which is how those audio features were actually calculated. The FMA includes the actual raw audio, so (while this is more of a longterm project than a stretch goal, and won't fit in Colab) if you'd like you can check those out and see what sort of deeper analysis you can do.